# Fine-Tuning Gemma-3-270m-it for Financial Sentiment Analysis

This notebook fine-tunes the `mlx-community/gemma-3-270m-it-bf16` model on a financial sentiment analysis task using MLX-LM with LoRA.

## Overview
- **Model**: gemma-3-270m-it-bf16 (~270M parameters)
- **Task**: Classify financial text as positive, negative, neutral, or bullish
- **Method**: LoRA (Low-Rank Adaptation)
- **Data Split**: 80% train / 10% validation / 10% test

## Hardware
- MacBook Pro M4 Max
- 40-core GPU
- 64GB unified memory

## 1. Setup and Configuration

In [ ]:
import pandas as pd
import json
import random
from pathlib import Path
from mlx_lm import load, generate
import mlx.core as mx

# Set random seed for reproducibility
random.seed(42)
mx.random.seed(42)

print("Setup complete!")

In [ ]:
# Configuration
MODEL_NAME = "mlx-community/gemma-3-270m-it-bf16"
DATA_DIR = Path("data")
ADAPTER_PATH = Path("adapters")
SOURCE_CSV = "sentiment_training_data.csv"

# Create directories
DATA_DIR.mkdir(exist_ok=True)
ADAPTER_PATH.mkdir(exist_ok=True)

print(f"Model: {MODEL_NAME}")
print(f"Data directory: {DATA_DIR}")
print(f"Adapter path: {ADAPTER_PATH}")

## 2. Data Preprocessing

Load the sentiment dataset created in `01_data_analysis.ipynb` and convert to MLX-LM's conversational JSONL format with 80/10/10 split.

In [ ]:
# Load source data
df = pd.read_csv(SOURCE_CSV)

print(f"Loaded {len(df)} records")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nLabel distribution:")
print(df['assistant'].value_counts())

In [ ]:
# Preview sample data
print("Sample entries:")
for i, row in df.sample(3, random_state=42).iterrows():
    print("=" * 60)
    print(f"SYSTEM: {row['system'][:100]}..." if len(str(row['system'])) > 100 else f"SYSTEM: {row['system']}")
    print(f"USER: {row['user'][:200]}..." if len(row['user']) > 200 else f"USER: {row['user']}")
    print(f"ASSISTANT: {row['assistant']}")

In [ ]:
# Convert to conversational format
data = []
for _, row in df.iterrows():
    # Use system prompt if available, otherwise use a default
    system_content = row['system'].strip() if pd.notna(row['system']) and row['system'].strip() else "You are a financial sentiment analysis expert. Analyze the sentiment of the given financial text and classify it as positive, negative, neutral, or bullish."
    
    conversation = {
        "messages": [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": row['user']
            },
            {
                "role": "assistant",
                "content": row['assistant']
            }
        ]
    }
    data.append(conversation)

# Shuffle data
random.shuffle(data)

print(f"Converted {len(data)} records to conversational format")
print("\nExample:")
print(json.dumps(data[0], indent=2))

In [ ]:
# Split data: 80% train, 10% validation, 10% test
total = len(data)
train_end = int(total * 0.8)
valid_end = int(total * 0.9)

train_data = data[:train_end]
valid_data = data[train_end:valid_end]
test_data = data[valid_end:]

print(f"Train: {len(train_data)} records ({len(train_data)/total:.0%})")
print(f"Valid: {len(valid_data)} records ({len(valid_data)/total:.0%})")
print(f"Test:  {len(test_data)} records ({len(test_data)/total:.0%})")

In [ ]:
# Save splits to JSONL files
splits = {
    'train.jsonl': train_data,
    'valid.jsonl': valid_data,
    'test.jsonl': test_data
}

for filename, split_data in splits.items():
    filepath = DATA_DIR / filename
    with open(filepath, 'w') as f:
        for entry in split_data:
            f.write(json.dumps(entry) + '\n')
    print(f"Saved {filepath} ({len(split_data)} records)")

print("\nData preprocessing complete!")

## 3. Fine-Tuning with LoRA

### Hyperparameters:
- **batch_size**: 8 — Number of samples processed per training iteration.
- **learning_rate**: 5e-5 — Step size for gradient descent optimization.
- **lr_schedule**: cosine_decay with 10% warmup — Learning rate warms up then decays following a cosine curve.
- **iters**: 1800 — Total number of training iterations.
- **num_layers**: 18 — Number of transformer layers to apply LoRA adapters to.
- **lora_rank**: 16 — Rank of the low-rank decomposition matrices.
- **lora_scale**: 2.0 — Scaling factor applied to LoRA outputs (alpha/rank).
- **lora_dropout**: 0.1 — Dropout probability for regularization.
- **val_batches**: 25 — Number of batches used for validation evaluation.
- **save_every**: 100 — Checkpoint saving frequency in iterations.

In [ ]:
# Training configuration is defined in lora_config.yaml
# This allows LoRA parameters (rank, scale, dropout) to be configured properly

CONFIG_PATH = Path("lora_config.yaml")

# Display the config
print("Training configuration (from lora_config.yaml):")
print("-" * 50)
print(CONFIG_PATH.read_text())

In [ ]:
# Run LoRA fine-tuning
import subprocess

print("Starting LoRA fine-tuning...\n")

# Use config file which includes lora_parameters (rank, scale, dropout)
cmd = [
    "python", "-m", "mlx_lm", "lora",
    "--config", str(CONFIG_PATH),
]

print(f"Command: {' '.join(cmd)}\n")

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("\nFine-tuning completed successfully!")
else:
    print(f"\nFine-tuning failed with return code {result.returncode}")